# Image Classification with CNN: Performance Analysis under Manipulations and Color Constancy

Bu projede, hayvanlara ait görüntüleri sınıflandırmak için bir Convolutional Neural Network (CNN) modeli tasarlanmış ve eğitilmiştir. Çalışma, görüntü sınıflandırma alanında CNN'in temel prensiplerini uygularken farklı test koşulları altında modelin performansını analiz etmeyi amaçlamaktadır.

Proje kapsamında, Kaggle'da yer alan "Animals with Attributes 2" veri seti kullanılmıştır. Bu veri setinden 10 farklı hayvan sınıfı seçilmiş, görüntüler üzerinde veri ön işleme adımları gerçekleştirilmiştir. Model eğitildikten sonra, test seti manipülasyonlara (ışık değişimleri ve bulanıklaştırma) tabi tutulmuş, ardından renk sabitliği algoritması uygulanarak performans değerlendirmesi yapılmıştır.

Sonuç olarak, normal test seti, manipüle edilmiş test seti ve renk sabitliği uygulanmış test seti üzerindeki başarı oranları karşılaştırılmış ve modelin dayanıklılığı hakkında çıkarımlar yapılmıştır.

**İçindekiler**

**1. Veri Setinin Kullanımı**
   
* Veri Seti Seçimi ve Sınıfların Belirlenmesi
* Görüntülerin İşlenmesi ve Normalize Edilmesi

**2. Veri Setinin Hazırlanması**

* Etiketlerin Kodlanması
* Eğitim ve Test Setlerine Ayırma

**3. CNN Modelinin Tasarımı**

* Model Mimarisi
* Kullanılan Katmanlar ve Parametreler

**4. Modelin Eğitilmesi**

* Eğitim Süreci ve Veri Artırma Teknikleri

**5. Modelin Test Edilmesi**

* Normal Test Seti Performansı

**6. Resimlerin Manipüle Edilmesi**

* Parlaklık Değişimi ve Bulanıklaştırma Teknikleri
* Manipüle Edilmiş Test Setinin Oluşturulması

**7. Manipüle Edilmiş Test Seti ile Performans Değerlendirme**

* Manipülasyon Altında Model Performansı

**8. Renk Sabitliği Algoritmasının Uygulanması**

* Gray World Algoritmasının Kullanımı
* Renk Sabitliği Uygulanmış Test Setinin Hazırlanması

**9. Renk Sabitliği Altında Performans Değerlendirme**

* Modelin Renk Sabitliği Uygulanmış Test Seti Üzerindeki Başarısı

**10. Sonuçların Karşılaştırılması ve Raporlama**

* Normal, Manipüle Edilmiş ve Renk Sabitliği Uygulanmış Test Setleri Arasındaki Karşılaştırma
Çıkarımlar ve Öneriler

# 1. Veri Setini Kullanma
Bu bölümde, sınıflandırma için kullanılacak veri seti seçilmiş ve veri setiyle ilgili gerekli düzenlemeler yapılmıştır. Animals with Attributes 2 adlı veri seti, Kaggle üzerinden elde edilmiştir. Bu veri setinde çeşitli hayvan türlerine ait binlerce görüntü bulunmaktadır. Projede kullanılacak 10 hayvan sınıfı belirlenmiş ve her sınıf için 650 görüntü seçilmiştir.

Seçilen görüntüler, modelin giriş katmanıyla uyumlu olacak şekilde yeniden boyutlandırılmış ve normalize edilmiştir. Normalize işlemi, tüm piksel değerlerinin 0 ile 1 arasında ölçeklenmesini sağlamıştır. Bu adımlar, modelin daha verimli ve doğru şekilde öğrenmesini sağlamak için gerçekleştirilmiştir.

In [11]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="keras.src.trainers.data_adapters.py_dataset_adapter")

# Gerekli kütüphaneleri yükleme
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

# Veri seti dizinleri
data_dir = "/kaggle/input/animals-with-attributes-2/Animals_with_Attributes2/JPEGImages"
selected_classes = ['collie', 'dolphin', 'elephant', 'fox', 'moose', 'rabbit', 'sheep', 'squirrel', 'giant+panda', 'polar+bear']
image_size = (128, 128)
max_images_per_class = 650

# Veri ve etiketleri yükleme
X, y = [], []
for class_name in selected_classes:
    class_dir = os.path.join(data_dir, class_name)
    images = os.listdir(class_dir)[:max_images_per_class]
    for img_name in tqdm(images, desc=f"Processing {class_name}"):
        img_path = os.path.join(class_dir, img_name)
        img = cv2.imread(img_path)
        img_resized = cv2.resize(img, image_size)
        X.append(img_resized / 255.0)  # Normalize et
        y.append(class_name)

# Veriyi numpy dizilerine dönüştürme
X = np.array(X)
y = np.array(y)

Processing polar+bear: 100%|██████████| 650/650 [00:08<00:00, 73.72it/s]


# 2. Veri Setini Hazırlama
Bu bölümde, veri seti eğitim ve test seti olarak ikiye ayrılmıştır. Eğitim seti modelin öğrenmesini sağlarken, test seti modelin daha önce görmediği veriler üzerinde performansını değerlendirmek için kullanılmıştır.

Etiketler, model tarafından işlenebilir hale getirilmek için sayısal değerlere dönüştürülmüştür (label encoding). Eğitim ve test setleri, veri çeşitliliğini sağlamak için rastgele seçilmiştir. Ancak bu rastgele seçimin her çalıştırmada aynı sonucu vermesi için sabit bir random_state değeri kullanılmıştır.

Eğitim setinde veri artırma (data augmentation) teknikleri uygulanmıştır. Bu, modelin daha genel bir şekilde öğrenmesini sağlamak ve overfitting (aşırı öğrenme) problemini önlemek için yapılmıştır.

In [12]:
# Etiketleri kodlama
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Veriyi eğitim ve test setine ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.3, random_state=42)

# 3. CNN Modelinin Tasarlanması
Bu bölümde, hayvan sınıflarını ayırt edebilmek için bir Convolutional Neural Network (CNN) modeli tasarlanmıştır. CNN, görüntü verileri üzerinde etkili sonuçlar veren, derin öğrenme alanında sıkça kullanılan bir modeldir.

Model üç temel katmandan oluşmaktadır:

Konvolüsyon Katmanları: Görüntüdeki özellikleri çıkarır ve bu özellikleri sonraki katmanlara iletir.

Havuzlama Katmanları: Modelin daha az parametreyle çalışmasını sağlamak için veri boyutunu azaltır.
Fully Connected Katmanlar: Sınıflandırmayı gerçekleştirir.

Dropout katmanları da eklenerek, aşırı öğrenme problemini önlemek hedeflenmiştir. Modelin çıktısı, 10 sınıfı temsil eden bir softmax katmanı ile sağlanmıştır.

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input

# CNN modeli
model = Sequential([
    Input(shape=(128, 128, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 4. Modelin Eğitilmesi
Bu bölümde, model seçilen eğitim verisi üzerinde eğitilmiştir. Eğitim sırasında veri artırma teknikleri uygulanarak modelin farklı durumlarla başa çıkma yeteneği artırılmıştır.
Eğitim süreci boyunca:

Model, her epoch'ta eğitim verisi üzerinde güncellenmiştir.
Eğitim ve doğrulama setlerinde kayıp (loss) ve doğruluk (accuracy) oranları izlenmiştir.
Modelin performansı, validation setindeki sonuçlara göre değerlendirilmiştir.
Eğitim sırasında kullanılan veri artırma teknikleri arasında döndürme, boyut değiştirme, yatay çevirme ve parlaklık değişiklikleri yer almıştır.

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Veri artırma
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)
datagen.fit(X_train)

# Modeli eğitme
history = model.fit(datagen.flow(X_train, y_train, batch_size=32), validation_data=(X_test, y_test), epochs=20)

Epoch 1/20
143/143 ━━━━━━━━━━━━━━━━━━━━ 85s 578ms/step - accuracy: 0.2039 - loss: 2.1307 - val_accuracy: 0.4077 - val_loss: 1.7499
Epoch 2/20
143/143 ━━━━━━━━━━━━━━━━━━━━ 81s 560ms/step - accuracy: 0.3532 - loss: 1.8111 - val_accuracy: 0.4174 - val_loss: 1.6617
Epoch 3/20
143/143 ━━━━━━━━━━━━━━━━━━━━ 82s 570ms/step - accuracy: 0.3982 - loss: 1.6582 - val_accuracy: 0.5179 - val_loss: 1.4153
Epoch 4/20
143/143 ━━━━━━━━━━━━━━━━━━━━ 82s 566ms/step - accuracy: 0.4638 - loss: 1.4857 - val_accuracy: 0.5210 - val_loss: 1.3658
Epoch 5/20
143/143 ━━━━━━━━━━━━━━━━━━━━ 85s 585ms/step - accuracy: 0.4675 - loss: 1.5051 - val_accuracy: 0.5231 - val_loss: 1.3944
Epoch 6/20
143/143 ━━━━━━━━━━━━━━━━━━━━ 83s 571ms/step - accuracy: 0.4953 - loss: 1.4141 - val_accuracy: 0.5605 - val_loss: 1.3084
Epoch 7/20
143/143 ━━━━━━━━━━━━━━━━━━━━ 142s 571ms/step - accuracy: 0.5108 - loss: 1.3736 - val_accuracy: 0.5441 - val_loss: 1.2823
Epoch 8/20
143/143 ━━━━━━━━━━━━━━━━━━━━ 82s 565ms/step - accuracy: 0.5449 - loss: 

# 5. Modelin Test Edilmesi
Bu bölümde, model daha önce görmediği test seti ile test edilmiştir. Test sürecinde modelin doğruluk oranı ve kayıp değerleri hesaplanmıştır. Bu değerlendirme, modelin performansını ölçmek için kritik bir adımdır.

Test sonuçları, modelin hangi sınıflarda daha iyi veya daha kötü performans gösterdiğini analiz etmek için de kullanılmıştır. Başarı oranı, modelin gerçek hayatta karşılaşabileceği veri çeşitliliğine nasıl tepki verdiğini gösterir.

In [15]:
# Test seti ile model performansı
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

61/61 ━━━━━━━━━━━━━━━━━━━━ 8s 132ms/step - accuracy: 0.6713 - loss: 0.9647
Test Loss: 0.9217363595962524, Test Accuracy: 0.6969230771064758


# 6. Resimlerin Farklı Işıklar ile Manipüle Edilmesi
Bu bölümde, test verilerine çeşitli manipülasyonlar uygulanmıştır. Manipülasyon işlemleri, modelin gerçek hayatta karşılaşabileceği zorlu koşullara dayanıklılığını test etmek için yapılmıştır.

Uygulanan manipülasyonlar:

Parlaklık Değişimi: Görüntülerin parlaklığı artırılarak aydınlatma koşullarındaki değişim simüle edilmiştir.

Bulanıklaştırma: Görüntüler, bulanıklaştırılarak detayların kaybolması durumunda modelin performansı test edilmiştir.

Bu manipülasyonlar sonucunda oluşturulan yeni test seti, modelin dayanıklılığını değerlendirmek için kullanılmıştır.


In [16]:
# Manipülasyon fonksiyonları
def manipulate_image(img, brightness_factor=1.5, blur_kernel=(5, 5)):
    # Parlaklık artırma
    bright_img = cv2.convertScaleAbs(img, alpha=brightness_factor, beta=0)
    # Bulanıklaştırma
    blurred_img = cv2.GaussianBlur(bright_img, blur_kernel, 0)
    return blurred_img

# Manipülasyonlu test seti
manipulated_X_test = np.array([manipulate_image(img) for img in X_test])

# 7. Modelin Manipüle Edilmiş Test Seti ile Denenmesi
Manipüle edilmiş test seti, modele verilerek performansı değerlendirilmiştir. Bu aşamada, manipülasyonların modelin doğruluk oranını nasıl etkilediği gözlemlenmiştir.

Sonuçlar, modelin manipülasyonlar karşısındaki dayanıklılığını ve zayıf yönlerini belirlemek için kullanılmıştır. Eğer manipülasyonlar sonrası doğruluk oranı önemli ölçüde düşmüşse, bu durum modelin gerçek dünyadaki kullanımı için önemli ipuçları sağlamıştır.

In [17]:
# Manipüle edilmiş test seti ile model performansı
manipulated_test_loss, manipulated_test_accuracy = model.evaluate(manipulated_X_test, y_test)
print(f"Manipulated Test Loss: {manipulated_test_loss}, Accuracy: {manipulated_test_accuracy}")

61/61 ━━━━━━━━━━━━━━━━━━━━ 9s 143ms/step - accuracy: 0.3626 - loss: 3.7163
Manipulated Test Loss: 3.6486458778381348, Accuracy: 0.3692307770252228


# 8. Manipüle Edilmiş Test Setine Renk Sabitliği Algoritması Uygulanması
Bu bölümde, manipüle edilmiş görüntülere renk sabitliği algoritması uygulanmıştır. Gray World Algoritması, görüntülerin renk dengesini düzeltmek için kullanılmıştır.

Gray World Algoritması, her bir renk kanalının ortalamasını eşitleyerek renk dengesizliklerini azaltır. Bu sayede, manipülasyonlar nedeniyle bozulan görüntülerdeki renk bilgileri daha tutarlı hale getirilmiştir.

In [18]:
def apply_gray_world(img):
    avg_b = np.mean(img[:, :, 0])
    avg_g = np.mean(img[:, :, 1])
    avg_r = np.mean(img[:, :, 2])
    avg_gray = (avg_b + avg_g + avg_r) / 3
    
    # Çok küçük ortalamalar için ölçeklendirme faktörü
    scale_factor = 1e-6  
    img[:, :, 0] = img[:, :, 0] * (avg_gray / (avg_b + scale_factor))
    img[:, :, 1] = img[:, :, 1] * (avg_gray / (avg_g + scale_factor))
    img[:, :, 2] = img[:, :, 2] * (avg_gray / (avg_r + scale_factor))
    
    return np.clip(img, 0, 255)

# Renk sabitliği uygulanmış test seti
gray_world_test_set = np.array([apply_gray_world(img.copy()) for img in manipulated_X_test])
gray_world_test_set = gray_world_test_set / 255.0  # Normalize

# 9. Modelin Renk Sabitliği Uygulanmış Test Seti ile Denenmesi
Renk sabitliği uygulanmış test seti modele verilerek performansı yeniden değerlendirilmiştir. Bu aşamada, renk sabitliği algoritmasının modelin doğruluk oranını nasıl etkilediği gözlemlenmiştir.

Elde edilen sonuçlar, renk sabitliğinin manipülasyonların etkilerini azaltmada ne kadar etkili olduğunu ortaya koymuştur.

In [19]:
# Renk sabitliği uygulanmış test seti ile model performansı
gray_world_test_loss, gray_world_test_accuracy = model.evaluate(gray_world_test_set, y_test)
print(f"Gray World Test Loss: {gray_world_test_loss}, Accuracy: {gray_world_test_accuracy}")

61/61 ━━━━━━━━━━━━━━━━━━━━ 8s 131ms/step - accuracy: 0.0991 - loss: 2.9985
Gray World Test Loss: 3.007535457611084, Accuracy: 0.1035897433757782


# 10. Farklı Test Setlerinin Başarılarını Karşılaştırma ve Raporlama
Bu bölümde, modelin normal, manipüle edilmiş ve renk sabitliği uygulanmış test setlerindeki doğruluk oranları karşılaştırılmıştır.
Karşılaştırma sonucunda:

Manipülasyonların model performansını ne ölçüde etkilediği,
Renk sabitliği algoritmasının manipülasyonların etkisini azaltmadaki başarısı analiz edilmiştir.
Sonuçlara dayanarak modelin daha dayanıklı hale getirilmesi için önerilerde bulunulmuştur.

In [20]:
# Sonuçların karşılaştırılması
print(f"""
Normal Test Set Accuracy: {test_accuracy}
Manipulated Test Set Accuracy: {manipulated_test_accuracy}
Gray World Test Set Accuracy: {gray_world_test_accuracy}
""")


Normal Test Set Accuracy: 0.6969230771064758
Manipulated Test Set Accuracy: 0.3692307770252228
Gray World Test Set Accuracy: 0.1035897433757782



Projedeki oranların düşük çıkmasının temel nedeni, modelin manipüle edilmiş ve renk sabitliği uygulanmış görüntülerle eğitilmemiş olmasıdır. Modelin dayanıklılığını artırmak için manipülasyonlu ve renk sabitliği uygulanmış görüntülerle eğitilmesi veya farklı renk sabitliği algoritmalarının test edilmesi önerilir. Ayrıca transfer öğrenimi gibi gelişmiş teknikler, model performansını daha da artırabilir.

Bu düzenlemelerle, modelin hem normal hem de manipüle edilmiş veriler üzerindeki doğruluk oranları iyileştirilebilir. Eğer bu adımlar sonucunda performansta anlamlı bir artış gözlemlenirse, modelin gerçek dünya uygulamaları için daha uygun hale gelmesi sağlanabilir.